<a href="https://colab.research.google.com/github/tackulus/229352/blob/main/Lab09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Lab 09**

---

> **229352 Statistical Learning for Data Science 2**

> **Kasidis Torcharoen (610510531)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip /content/drive/MyDrive/pokemon.zip;
!mkdir /content/drive/MyDrive/new_pokemon; #Folder to save images of new pokemon
!mkdir /content/drive/MyDrive/GAN_weights #Folder to save models' weights

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.layers import Input, Flatten, Reshape, RandomRotation
from tensorflow.keras.models import Model
from tensorflow.image import random_hue

In [ ]:
#TODO-0: set the training parameters below
BATCH_SIZE = 16
EPOCHS = 80
noise_dim = 100 


num_examples_to_generate = 16
IMAGE_SIZE = 64


images = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/content/pokemon/pokemon', label_mode=None,
    class_names=None, color_mode='rgb', batch_size=BATCH_SIZE, image_size=(IMAGE_SIZE,
    IMAGE_SIZE), shuffle=True)

#preprocessing+augmentation
images = images.map(lambda x: random_hue(x,0.5))
images = images.map(lambda x: (x-127.5)/127.5) #transform pixels to [-1,1] 
images = images.cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


Found 15467 files belonging to 1 classes.


In [ ]:
#Generator transforms a Gaussian vector into a 64x64x3 image

z = Input(shape=(noise_dim,)) 

x = Dense(64*8*4*4, use_bias=False)(z)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Reshape((4,4,64*8))(x)

# see https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose
x = Conv2DTranspose(256, 4 , strides=2, padding='same' , use_bias = False)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

#TODO-1: fill the rest of the model
x = Conv2DTranspose(128, 4 , strides=2, padding='same' , use_bias = False)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2DTranspose(64, 4 , strides=2, padding='same' , use_bias = False)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

#TODO-1
out = Conv2DTranspose(3, 4 , strides=2, padding='same' , use_bias = False, 
                      activation='tanh')(x)

generator = Model(inputs = z, outputs = out)


In [ ]:
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense (Dense)                (None, 8192)              819200    
_________________________________________________________________
batch_normalization (BatchNo (None, 8192)              32768     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 8192)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 256)         2097152   
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 256)         1024  

In [ ]:
#Discrimination takes an image, classifies whether the image is real or fake

inp = Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3))

y = RandomRotation(0.05, fill_mode='constant',fill_value=1.0)(inp)

y = Conv2D(64, 4, strides = 2, padding = 'same', use_bias = False)(y)
y = BatchNormalization()(y)
y = LeakyReLU(0.2)(y)

#TODO-2: fill the rest of the model
y = Conv2D(128, 4, strides = 2, padding = 'same', use_bias = False)(y)
y = BatchNormalization()(y)
y = LeakyReLU(0.2)(y)

y = Conv2D(256, 4, strides = 2, padding = 'same', use_bias = False)(y)
y = BatchNormalization()(y)
y = LeakyReLU(0.2)(y)

y = Conv2D(512, 4, strides = 2, padding = 'same', use_bias = False)(y)
y = BatchNormalization()(y)
y = LeakyReLU(0.2)(y)

#TODO-2
y = Flatten()(y)
y_out = Dense(1, activation='sigmoid')(y)

discriminator = Model(inputs = inp, outputs = y_out)

In [ ]:
discriminator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        3072      
_________________________________________________________________
batch_normalization_4 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       131072    
_________________________________________________________________
batch_normalization_5 (Batch (None, 16, 16, 128)       512 

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_pred, fake_pred): 
    #TODO-3: Modify the following line to apply Label Smoothing
    real_loss = cross_entropy(tf.ones_like(real_pred) * 0.9, real_pred)
    fake_loss = cross_entropy(tf.zeros_like(fake_pred), fake_pred)
    total_loss = (real_loss + fake_loss)
    return total_loss
def generator_loss(fake_pred):
    return cross_entropy(tf.ones_like(fake_pred), fake_pred)

In [ ]:
# Setup Adam optimizers for both Generator and Discriminator

#TODO-4: specify optimizers of the generator and discriminator
# generator: Adam with learning rate = 0.0002 and beta_1 = 0.5
# discriminator: Adam with learning rate = 0.0004 and beta_1 = 0.5
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0004, beta_1=0.5)

In [ ]:
#No TODO here

def train_step(real_images): # real_images: minibatch of real images
    noise = tf.random.normal(mean=0, stddev=1, shape=(BATCH_SIZE, noise_dim))

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        #generate_data & classify as fake or real
        generated_images = generator(noise,training=True)

        real_pred = discriminator(real_images,training = True)
        fake_pred = discriminator(generated_images, training=True)

        #compute loss
        gen_loss = generator_loss(fake_pred)
        disc_loss = discriminator_loss(real_pred, fake_pred)

    #compute gradients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    #update parameters with gradient descent
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss


In [ ]:
# 16 random gaussian vectors to generate example images
noises = tf.random.normal(mean=0, stddev=1, shape=(num_examples_to_generate, noise_dim))

#TODO-5: Question: which line generates new pokemon from random noises?

for epoch in range(EPOCHS):
    print('Starting epoch ',epoch)
    for image_batch in tqdm(images):
        gen_loss, disc_loss =train_step(image_batch)

    print('generative loss: ',float(gen_loss),
          ' discriminative loss: ',float(disc_loss))

    predictions = generator(noises, training=False) # This line

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
       plt.subplot(4, 4, i+1)
       plt.imshow((predictions[i, :, :, :] +1)/2)
       plt.axis('off')

    plt.savefig(f'/content/drive/MyDrive/new_pokemon/image_at_epoch_{epoch:04d}.png')
    generator.save_weights(f'/content/drive/MyDrive/GAN_weights/pokemon_generator_{epoch:04d}')
    discriminator.save_weights(f'/content/drive/MyDrive/GAN_weights/pokemon_discriminator_{epoch:04d}')

Starting epoch  0


100%|██████████| 967/967 [03:15<00:00,  4.96it/s]


generative loss:  1.3204978704452515  discriminative loss:  0.8154924511909485
Starting epoch  1


100%|██████████| 967/967 [02:21<00:00,  6.85it/s]


generative loss:  1.0291776657104492  discriminative loss:  0.9107860326766968
Starting epoch  2


...

100%|██████████| 967/967 [02:32<00:00,  6.35it/s]


generative loss:  4.872702598571777  discriminative loss:  0.3813190460205078
Starting epoch  73


100%|██████████| 967/967 [03:21<00:00,  4.79it/s]


generative loss:  4.429702281951904  discriminative loss:  0.3871072828769684
Starting epoch  74


100%|██████████| 967/967 [03:21<00:00,  4.79it/s]

My program didn't finish on epoch 80. It maybe because of internet disconection.

In [ ]:
%%shell
jupyter nbconvert --to html /content/Lab09.ipynb

[NbConvertApp] Converting notebook /content/Lab09.ipynb to html
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed (u'metadata' was unexpected)

Failed validating u'additionalProperties' in stream:

On instance[u'cells'][11][u'outputs'][0]:
{u'metadata': {u'tags': None},
 u'name': u'stdout',
 u'output_type': u'stream',
 u'text': u'Starting epoch  0\n'}
[NbConvertApp] Writing 347357 bytes to /content/Lab09.html
